In [1]:
import spacy
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [5]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
class Text:
    def __init__(self, path, title=""):
        self.title = title
        self.path = path
        self.text = open(self.path, encoding='utf-8').read()
        self.word_count = 0
        self.sentence_count = 0
        self.dialogue_word_count = 0
        self.parag_count = 0
        self.vocab_count = 0
        self.generate_stats()
        
    def generate_stats(self):
        chunks = self.partition_text()
        unique_words = set()
        for i in chunks:
            doc = nlp(i, disable=['NER'])
            words, diag_words = self.count_words(doc)
            self.word_count += words
            self.dialogue_word_count += diag_words
            self.sentence_count += len(list(doc.sents))
            self.parag_count += self.parag_counter(doc)
            unique_words = unique_words.union(set([i.lemma_ for i in doc if not i.is_punct and i.text not in ["\n", "\n\n"]]))
            del doc
        self.vocab_count = len(unique_words)
    
    # NEEDS OVERHAUL
    def count_words(self, doc):
        """Takes a spacy doc and returns a 2-tuple of overall word count and dialogue word count."""
        words_with_quotations = [token.text for token in doc if 
                                 (not token.is_punct or token.text=='"' or token.text=="“" 
                                  or token.text=="”") and token.text!="\n\n" or token.text!="\n"]
        word_count = 0
        diag_count = 0
        diag = False
        thing = []
        for i in words_with_quotations:
            if i in ['"', "“", "”"]:
                diag = not diag
                continue
            if diag:
                diag_count += 1
                thing.append(i) #get rid of thing stuff
            word_count += 1
        print(thing[0:1000])
        return (word_count, diag_count)
    
    def partition_text(self):
        chunks = []
        while len(self.text) > 60000:
            split_at = 50000
            while self.text[split_at] != ".":
                split_at += 1
            chunks.append(self.text[0:split_at+1])
            self.text = self.text[split_at+1:]
        chunks.append(self.text)
        return chunks
    
    def parag_counter(self, doc):
        parag_count = 1
        for i in [token.text for token in doc]:
            if i == "\n\n" or i == "\n":
                parag_count += 1
        return parag_count

In [37]:
titles = ["The Speedrunner and the Kid", "The Order of the Bell", "Gravity's Rainbow"]#, "The Left Hand of Darkness",
         #"The Shining"]
files = ["C:/Users/awitt/Desktop/spd_rnr.txt", "C:/Users/awitt/Desktop/ordr_bll.txt", "C:/Users/awitt/Desktop/grv_rnb.txt"]
         #, "C:/Users/awitt/Desktop/lft_hnd.txt", 
         #"C:/Users/awitt/Desktop/shng.txt"]

In [48]:
title = ["grav"]
files = ["C:/Users/awitt/Desktop/grv_rnb.txt"]
books = [Book(i, j) for (i, j) in zip(titles, files)]

['You', 'did', 'n’t', 'really', 'believe', 'you', '’d', 'be', 'saved', '.', 'Come', ',', 'we', 'all', 'know', 'who', 'we', 'are', 'by', 'now', '.', 'No', 'one', 'was', 'ever', 'going', 'to', 'take', 'the', 'trouble', 'to', 'save', 'you', ',', 'old', 'fellow', '.', '.', '.', '.', 'Pirate', 'Good', 'morning', ',', 'Hhahh', ',', 'hhaahhh', '!', 'Incoming', 'mail', '.', 'Kill', 'myself', ',', 'The', 'Germans', 'will', 'do', 'it', 'for', 'you', '.', 'Guess', 'what', 'I', 'saw', 'from', 'the', 'roof', '.', 'That', 'V-2', 'on', 'the', 'way', '?', 'A4', ',', 'yes', '.', 'I', 'watched', 'it', 'out', 'the', 'window', '.', 'About', 'ten', 'minutes', 'ago', '.', 'Looked', 'queer', ',', 'did', 'n’t', 'it', '.', 'Have', 'n’t', 'heard', 'a', 'thing', 'since', ',', 'have', 'you', '.', 'It', 'must', 'have', 'fallen', 'short', '.', 'Out', 'to', 'sea', 'or', 'something', '.', 'Ten', 'minutes', '?', 'At', 'least', '.', 'Prentice', 'here', ',', 'did', 'you', 'have', 'anything', 'like', 'a', 'pip', 'from', 

KeyboardInterrupt: 

In [39]:
for i in books:
    print(i.title)
    print(i.word_count)
    print(i.sentence_count)
    print(i.dialogue_word_count / i.word_count)
    print(i.vocab_count)

The Speedrunner and the Kid
66850
6909
0.48463724756918475
4497
The Order of the Bell
142265
18096
0.44791058939303413
6968
Gravity's Rainbow
414955
28840
0.47942547987131134
20844


In [177]:
# Number of sentences in each book
spd_num_sents = len(list(spd_doc.sents))
ordr_num_sents = len(list(ordr_doc.sents))
print("spd has " + str(spd_num_sents) + " sentences")
print("ordr has " + str(ordr_num_sents) + " sentences")

spd has 6909 sentences
ordr has 18098 sentences


In [176]:
# Number of words in each book
spd_words = [token for token in spd_doc if not token.is_punct]
ordr_words = [token for token in ordr_doc if not token.is_punct]
spd_num_words = len(spd_words)
ordr_num_words = len(ordr_words)
spd_num_pages = len(spd_words)//300
ordr_num_pages = len(ordr_words)//300
print("spd has " + str(spd_num_words) + " words")
print("ordr has " + str(ordr_num_words) + " words")
print("spd has " + str(spd_num_pages) + " pages")
print("ordr has " + str(ordr_num_pages) + " pages")

spd has 58026 words
ordr has 123002 words
spd has 193 pages
ordr has 410 pages


In [163]:
# Number of words per sentence
spd_words_per_sentence = round(len(spd_words)/len(list(spd_doc.sents)), 2)
ordr_words_per_sentence = round(len(ordr_words)/len(list(ordr_doc.sents)), 2)
print("spd has " + str(spd_words_per_sentence) + " words per sentence")
print("ordr has " + str(ordr_words_per_sentence) + " words per sentence")

spd has 8.4 words per sentence
ordr has 6.8 words per sentence


In [99]:
spd_words = [token.text for token in spd_doc if not token.is_punct]
ordr_words = [token.text for token in ordr_doc if not token.is_punct]
spd_word_freq = Counter(spd_words)
ordr_word_freq = Counter(ordr_words)

In [151]:
from wordcloud import STOPWORDS
shit_stops = ['cee', '\n\n', '\n', "'s", 'n’t', '’s', "'d", "'m", "'re", '’d', '’re', '’m', "'ve", "'ll", '’ll', '’ve']
spd_stops = ['Macuahuitl', 'worldtree', 'felixthebeast', "n't", "Nikolai", "Gard", "re", "TlalocsKid", "ll", "Monica", "said", 've', 'EvilMollusc', 'WorldTree54', 'Reidar']
ordr_stops = ["Marto", "Claire", "Wendell", "Deborah", "Ben", "Alex", "Finch", "John", "re", "ve", "ll", "Mephisto", "said", "Khiver", "Adrienne", "Khemenehadra"]

In [152]:
spd_ed_words = [token.text.lower() for token in spd_doc if not token.is_punct and not token.is_stop]
spd_ed_words = [i for i in spd_ed_words if i not in [j.lower() for j in spd_stops] and i not in STOPWORDS and i not in shit_stops]
ordr_ed_words = [token.text.lower() for token in ordr_doc if not token.is_punct and not token.is_stop]
ordr_ed_words = [i for i in ordr_ed_words if i not in [j.lower() for j in ordr_stops] and i not in STOPWORDS and i not in shit_stops]

In [181]:
spd_freq_df = pd.DataFrame(list(Counter(spd_ed_words).most_common(25)), columns=["Word", "Uses"])
spd_freq_df["Uses per Page"] = round(spd_freq_df["Uses"]/spd_num_pages, 2)
spd_freq_df.index +=1

ordr_freq_df = pd.DataFrame(list(Counter(ordr_ed_words).most_common(25)), columns=["Word", "Uses"])
ordr_freq_df["Uses per Page"] = round(ordr_freq_df["Uses"]/ordr_num_pages, 2)
ordr_freq_df.index += 1

In [166]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [182]:
display(spd_freq_df)
display(ordr_freq_df)

,Word,Uses,Uses per Page
1,time,171,0.89
2,kid,150,0.78
3,right,141,0.73
4,father,137,0.71
5,way,134,0.69
6,going,126,0.65
7,want,106,0.55
8,long,101,0.52
9,let,94,0.49
10,sure,93,0.48


,Word,Uses,Uses per Page
1,told,238,0.58
2,eyes,231,0.56
3,time,225,0.55
4,room,221,0.54
5,turned,209,0.51
6,hand,200,0.49
7,going,196,0.48
8,looked,184,0.45
9,door,184,0.45
10,right,174,0.42


In [110]:
# WORDCLOUD STUFF

'''
# spd generic wordcloud
spd_stops = ['Macuahuitl', 'worldtree', 'felixthebeast', "n't", "Nikolai", "Gard", "re", "TlalocsKid", "ll", "Monica", "said", 've', 'EvilMollusc', 'WorldTree54', 'Reidar']
spd_ed_cloud = WordCloud(max_words=60, stopwords=set(spd_stops)|STOPWORDS, width=800, height=400).generate(" ".join(spd_ed_words))
plt.imshow(spd_ed_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# ordr generic wordcloud
ordr_stops = ["Marto", "Claire", "Wendell", "Deborah", "Ben", "Alex", "Finch", "John", "re", "ve", "ll", "Mephisto", "said", "Khiver", "Adrienne", "Khemenehadra"]
ordr_ed_cloud = WordCloud(max_words=60, stopwords=set(ordr_stops)|STOPWORDS, width=800, height=400).generate(" ".join(ordr_ed_words))
plt.imshow(ordr_ed_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

spd_ed_cloud.to_file("C:/Users/awitt/Desktop/spd_cloud.png")
# https://imgur.com/a/l0o8YIs
ordr_ed_cloud.to_file("C:/Users/awitt/Desktop/ordr_cloud.png")
# https://imgur.com/a/0gfbJc1
'''